In [ ]:
# !pip install transformers[torch] accelerate datasets torch scikit-learn evaluate

In [ ]:
import json
import difflib
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import evaluate

In [3]:
# Carregar o arquivo JSON
with open('dislalia_dataset_sem_duplicatas.json') as f:
    examples = json.load(f) 

In [ ]:
def identificar_erro(correta, erro):
    s = difflib.SequenceMatcher(None, correta, erro)
    differ = s.get_opcodes()
    
    substituicao = omissao = acrescimo = False
    
    for tag, i1, i2, j1, j2 in differ:
        if tag == 'replace':
            substituicao = True
        elif tag == 'delete':
            omissao = True
        elif tag == 'insert':
            acrescimo = True
            
    if substituicao:
        return "Substituição"
    elif omissao:
        return "Omissão"
    elif acrescimo:
        return "Acréscimo"

# Aplicar a função aos exemplos
for example in examples:
    tipo_previsto = identificar_erro(example['correta'], example['erro'])
    print(f"Correta: {example['correta']} | Erro: {example['erro']} | Tipo Previsto: {tipo_previsto}")

In [5]:
# Dicionário para mapear tipos de erro para rótulos numéricos
label_mapping = {"Substituição": 0, "Omissão": 1, "Acréscimo": 2}

# Criar listas de palavras corretas, erros e rótulos
corretas = [ex['correta'] for ex in examples]
erros = [ex['erro'] for ex in examples]
labels = [label_mapping[ex['tipo_erro']] for ex in examples]

# Dividir os dados em treino e validação
train_texts, val_texts, train_labels, val_labels = train_test_split(erros, labels, test_size=0.2)

# Inicializar o tokenizer
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenizar os dados
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Criar datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

# Juntar os datasets em um DatasetDict
datasets = DatasetDict({"train": train_dataset, "validation": val_dataset})

### Configuração do Modelo BERT

In [ ]:
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

### Carregar a Metrica

In [ ]:
# Carregar a métrica de avaliação corretamente
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

### Treinamento do Modelo

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    compute_metrics=compute_metrics
)

trainer.train()

### Avaliando o Modelo

In [ ]:
eval_results = trainer.evaluate()
print(f"Resultados da Avaliação: {eval_results}")

### Testando com Exemplos

In [ ]:
def predict_error_type(model, tokenizer, erro):
    inputs = tokenizer(erro, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    
    tipo_erro = list(label_mapping.keys())[list(label_mapping.values()).index(prediction)]
    return tipo_erro

# Testar com um novo exemplo
erro_novo = "balata"  # Palavra errada para "barata"
tipo_predito = predict_error_type(model, tokenizer, erro_novo)
print(f"Erro: {erro_novo} | Tipo Predito: {tipo_predito}")